## La Liga Data analysis from Data of Understat

In [ ]:
import pandas as pd
df = pd.read_csv("/Users/minseobeom/Desktop/라리가 데이터 (데이터 분석 및 예측 독학용)/Understat_Data_scraping/la_liga_2014-2025_match_info.csv")

print(df.head())

   match_id                 date home_team            away_team  home_goals  \
0      5826  2014-08-23 18:00:00    Malaga        Athletic Club           1   
1      5827  2014-08-23 20:00:00   Sevilla             Valencia           1   
2      5828  2014-08-23 20:00:00   Granada  Deportivo La Coruna           2   
3      5829  2014-08-23 22:00:00   Almeria             Espanyol           1   
4      5830  2014-08-24 18:00:00     Eibar        Real Sociedad           1   

   away_goals  home_xg  away_xg  home_shots  away_shots  home_sot  away_sot  \
0           0     1.32     1.14          12          12         3         5   
1           1     1.17     1.75          12          11         3         1   
2           1     0.55     0.38          10           8         3         1   
3           1     0.98     0.40          19          12         6         2   
4           0     0.47     0.98          12          19         5         4   

   home_deep  away_deep  home_ppda  away_ppda  hom

## Basic EDA Visualization (Exploratory Data Analysis)(2014-2025) ##

- Goal: Understand overall league trends and data distribution

- Seasonal Average Goals per Match (시즌별 경기당 득점 추이)

- Home Win / Draw / Away Win Rate Changes (홈 승률 / 무승부율 / 원정 승률 변화)

- Difference in Expected Goals (xG) between Home and Away → Box Plot or Violin Plot (홈 vs 원정 기대득점(xG) 차이)


# 1. Load & Prepare / 데이터 로드·준비

In [ ]:
# Step 0) Imports & basic config / 라이브러리 임포트 및 기본 설정
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["axes.unicode_minus"] = False  # Fix minus sign rendering / 마이너스 기호 깨짐 방지


In [ ]:

# Load CSV
df = pd.read_csv("/Users/minseobeom/Desktop/라리가 데이터 (데이터 분석 및 예측 독학용)/Understat_Data_scraping/la_liga_2014-2025_match_info.csv")

# Convert date column to datetime
df["date"] = pd.to_datetime(df["date"])

# Season label as start year only (e.g., 2014-2015 → 2014)
# 시즌 라벨을 시작 연도만 표기 (예: 2014-2015 → 2014)
def make_season_year(y, m):
    return y if m >= 8 else y - 1

df["season"] = df.apply(lambda r: make_season_year(r["date"].year, r["date"].month), axis=1)

# Match result H/D/A
# 경기 결과 (H=홈승, D=무승부, A=원정승)
df["result"] = np.where(
    df["home_goals"] > df["away_goals"], "H",
    np.where(df["home_goals"] < df["away_goals"], "A", "D")
)

plt.rcParams["axes.unicode_minus"] = False  # minus sign rendering fix
print(df[["date", "home_team", "away_team", "home_goals", "away_goals", "season", "result"]].head())


                 date home_team            away_team  home_goals  away_goals  \
0 2014-08-23 18:00:00    Malaga        Athletic Club           1           0   
1 2014-08-23 20:00:00   Sevilla             Valencia           1           1   
2 2014-08-23 20:00:00   Granada  Deportivo La Coruna           2           1   
3 2014-08-23 22:00:00   Almeria             Espanyol           1           1   
4 2014-08-24 18:00:00     Eibar        Real Sociedad           1           0   

   season result  
0    2014      H  
1    2014      D  
2    2014      H  
3    2014      D  
4    2014      H  


# 2. Seasonal Average Goals per Match (시즌별 경기당 득점 추이)

In [ ]:
import pandas as pd
import plotly.express as px

# 시즌별 평균 득점 데이터프레임 예시
season_goals = df.groupby("season").apply(
    lambda x: (x["home_goals"] + x["away_goals"]).sum() / len(x)
).reset_index(name="goals_per_match")

# Plotly 라인 차트 생성
fig = px.line(
    season_goals,
    x="season",
    y="goals_per_match",
    markers=True,
    title="Goals per Match by Season",
    labels={"season": "Season / 시즌", "goals_per_match": "Goals per Match / 경기당 득점"},
    hover_data={"season": True, "goals_per_match": True}  # 마우스오버 시 표시할 데이터
)

# 그래프 표시
fig.show()


/var/folders/yl/3cpbqsq96s70dm9lcf8p9c_00000gn/T/ipykernel_26056/2911352031.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  season_goals = df.groupby("season").apply(


# 3. Home Win / Draw / Away Win Rate Changes (홈 승률 / 무승부율 / 원정 승률 변화)

In [ ]:
import plotly.graph_objects as go

# 1) 시즌별 결과 개수 → 비율
res_counts = (
    df.groupby(["season", "result"])
      .size()
      .unstack(fill_value=0)[["H","D","A"]]     # 고정 순서
)
res_rates = (res_counts.T / res_counts.sum(axis=1)).T  # 각 시즌 합 1.0

seasons = res_rates.index.astype(str).tolist()
H = res_rates["H"].values
D = res_rates["D"].values
A = res_rates["A"].values

# 2) 인터랙티브 스택드 바
fig = go.Figure()
fig.add_bar(
    x=seasons, y=H, name="Home Win / 홈승",
    hovertemplate="Season: %{x}<br>Home Win: %{y:.1%}<extra></extra>"
)
fig.add_bar(
    x=seasons, y=D, name="Draw / 무승부",
    hovertemplate="Season: %{x}<br>Draw: %{y:.1%}<extra></extra>"
)
fig.add_bar(
    x=seasons, y=A, name="Away Win / 원정승",
    hovertemplate="Season: %{x}<br>Away Win: %{y:.1%}<extra></extra>"
)

fig.update_layout(
    barmode="stack",
    title="Result Rates by Season (Interactive) / 시즌별 결과 비율",
    xaxis_title="Season / 시즌",
    yaxis_title="Rate / 비율",
)
fig.show()


3-1. Team Result Rates over Seasons / 팀별 결과 비율 (누적 막대)

In [ ]:
# ==========================================================
# Step 7B) Team Result Counts over Seasons — Interactive (Dropdown)
#          팀별 시즌 승/무/패 경기 수 — 인터랙티브(드롭다운)
# ==========================================================

import numpy as np
import pandas as pd
import plotly.graph_objects as go

# 0) 홈/원정 → 팀별 경기 결과 데이터프레임 만들기
home_rows = pd.DataFrame({
    "season": df["season"],
    "team": df["home_team"],
    "is_win":  (df["home_goals"] >  df["away_goals"]).astype(int),
    "is_draw": (df["home_goals"] == df["away_goals"]).astype(int),
    "is_loss": (df["home_goals"] <  df["away_goals"]).astype(int),
})
away_rows = pd.DataFrame({
    "season": df["season"],
    "team": df["away_team"],
    "is_win":  (df["away_goals"] >  df["home_goals"]).astype(int),
    "is_draw": (df["away_goals"] == df["home_goals"]).astype(int),
    "is_loss": (df["away_goals"] <  df["home_goals"]).astype(int),
})
team_match = pd.concat([home_rows, away_rows], ignore_index=True)

# 1) 시즌별 승/무/패 경기 수 집계
counts = (team_match
          .groupby(["season","team"], as_index=False)
          .agg(W=("is_win","sum"), D=("is_draw","sum"), L=("is_loss","sum")))

teams = sorted(counts["team"].unique().tolist())
seasons = sorted(counts["season"].unique().tolist())
seasons_str = list(map(str, seasons))

def get_team_series(t: str) -> pd.DataFrame:
    """Return season-indexed W/D/L counts for a team / 팀의 시즌별 승/무/패 경기 수"""
    s = (counts.loc[counts["team"].eq(t), ["season","W","D","L"]]
               .set_index("season")
               .reindex(seasons)
               .fillna(0))
    return s

# 2) 초기 팀 설정
init_team = teams[0]
init_series = get_team_series(init_team)
cd = np.array([init_team] * len(seasons_str), dtype=object)

fig = go.Figure(data=[
    go.Bar(x=seasons_str, y=init_series["W"], name="Win / 승",
           customdata=cd,
           hovertemplate="Team: %{customdata}<br>Season: %{x}<br>Win: %{y} 경기<extra></extra>"),
    go.Bar(x=seasons_str, y=init_series["D"], name="Draw / 무",
           customdata=cd,
           hovertemplate="Team: %{customdata}<br>Season: %{x}<br>Draw: %{y} 경기<extra></extra>"),
    go.Bar(x=seasons_str, y=init_series["L"], name="Loss / 패",
           customdata=cd,
           hovertemplate="Team: %{customdata}<br>Season: %{x}<br>Loss: %{y} 경기<extra></extra>")
])

# 3) 레이아웃: 드롭다운 위치, 상단 여백
fig.update_layout(
    barmode="stack",
    title=f"Result Counts by Season — {init_team} / 시즌 결과 경기 수 — {init_team}",
    xaxis_title="Season / 시즌",
    yaxis_title="Number of Matches / 경기 수",
    legend_title_text="Result / 결과",
    updatemenus=[dict(
        type="dropdown",
        buttons=[],
        x=1.0, y=1.18, xanchor="right", yanchor="top",
        showactive=True
    )],
    margin=dict(t=110)
)

# 4) 드롭다운 버튼 생성
buttons = []
for t in teams:
    s = get_team_series(t)
    cd_t = np.array([t] * len(seasons_str), dtype=object)
    buttons.append(dict(
        label=t,
        method="update",
        args=[
            {"y": [s["W"], s["D"], s["L"]],
             "customdata": [cd_t, cd_t, cd_t]},
            {"title": f"Result Counts by Season — {t} / 시즌 결과 경기 수 — {t}"}
        ]
    ))

fig.update_layout(updatemenus=[dict(
    type="dropdown",
    buttons=buttons,
    x=1.0, y=1.18, xanchor="right", yanchor="top",
    showactive=True
)])

fig.show()


# 4. Difference in Expected Goals (xG) between Home and Away → Box Plot or Violin Plot (홈 vs 원정 기대득점(xG) 차이)

4-1. Per-match xG Difference — Distribution (경기 단위 xG 차이 분포)

In [ ]:
import pandas as pd
import plotly.express as px

# xG difference per match = home_xg - away_xg
# 경기 단위 xG 차이 = 홈 xG - 원정 xG
df["xg_diff"] = (df["home_xg"] - df["away_xg"]).astype(float)

# Interactive histogram with a reference line at 0
# 0 기준선 포함 인터랙티브 히스토그램
fig = px.histogram(
    df.dropna(subset=["xg_diff"]),
    x="xg_diff",
    nbins=40,
    title="xG Difference per Match (Home - Away)",
    labels={"xg_diff": "xG Difference (Home - Away)"},
    marginal="box",  # add a small boxplot on the side / 옆에 박스플롯 추가
    opacity=0.8
)
fig.update_traces(hovertemplate="xG diff: %{x:.3f}<br>Count: %{y}<extra></extra>")
fig.add_vline(x=0, line_width=2, line_dash="dash", line_color="gray")  # reference line / 기준선
fig.show()


- English
    
    If the distribution is shifted to the right of 0, it means that in most matches, the home team generated more xG than the away team → a sign of home advantage.
    If the center (mean/median) of the distribution is greater than 0, it indicates that across the league, home teams generally create better chances than away teams.
- 한국어
    
    분포가 0보다 오른쪽으로 치우쳐 있으면, 대부분의 경기에서 홈팀의 기대득점(xG)이 원정보다 높았다는 의미 → 홈 어드밴티지 신호.
    분포의 중심(평균/중앙값)이 0보다 크면, 리그 전체적으로 홈팀이 원정팀보다 더 좋은 득점 기회를 만든다는 의미.

4-2. Seasonal Mean xG Difference — Trend (시즌별 평균 xG 차이 추세)

In [ ]:
import numpy as np
import plotly.express as px

# Aggregate by season: mean, std, count of xg_diff
# 시즌별 xg_diff의 평균/표준편차/표본수 집계
season_xgdiff = (df
    .dropna(subset=["xg_diff"])
    .groupby("season")
    .agg(mean_xg_diff=("xg_diff","mean"),
         std_xg_diff=("xg_diff","std"),
         n=("xg_diff","size"))
    .reset_index()
)

# Interactive line chart
# 인터랙티브 라인 차트
fig = px.line(
    season_xgdiff,
    x="season", y="mean_xg_diff",
    markers=True,
    title="Seasonal Mean xG Difference (Home - Away)",
    labels={"season":"Season","mean_xg_diff":"Mean xG Difference"},
    hover_data={"std_xg_diff":":.3f","n":True}
)
fig.add_hline(y=0, line_width=2, line_dash="dash", line_color="gray")  # reference line / 기준선
fig.update_traces(hovertemplate="Season: %{x}<br>Mean xG diff: %{y:.3f}<br>Std: %{customdata[0]:.3f}<br>N: %{customdata[1]}<extra></extra>")
fig.show()


- English

    If the line stays above 0, that season’s average home xG was higher than away xG.
    The slope or pattern of the line across seasons shows whether home advantage strength is increasing or decreasing over time.
- 한국어

    라인이 0 위에 있으면, 해당 시즌의 평균 홈팀 xG가 원정보다 높았다는 뜻.
    시즌 간 라인의 기울기나 패턴을 보면 홈 어드밴티지의 강도가 시간이 지남에 따라 강화되고 있는지, 약화되고 있는지를 알 수 있음.

In [ ]:
# 📊 Graph Interpretation / 그래프 해석
# ------------------------------------------------------------
# EN:
# X-axis (Season): From 2014 season to 2024 season (11 seasons)
# Y-axis (Mean xG Difference): Average of (Home xG − Away xG) for each season
# - Positive values: Home teams have higher expected goals than away teams on average
# - Zero baseline: Home and away teams have equal expected goals
# - Negative values: Away teams have higher expected goals than home teams (rare in this dataset)
#
# 📌 Key Observations:
# - 2014–2019 seasons: Mean difference ~0.38–0.50 → Consistent home advantage in xG
# - 2020 season: Drop to ~0.21 → Possible effect of COVID-19 no-audience matches
# - 2021–2023 seasons: Recovered to ~0.49 → Home advantage regained
# - 2024 season: Slight decline → Possible sign of reduced home advantage
#
# KO:
# X축(Season): 2014 시즌부터 2024 시즌까지 11시즌
# Y축(평균 xG 차이): 시즌별 (홈 xG − 원정 xG) 평균
# - 양수 값: 홈팀이 평균적으로 원정팀보다 기대득점이 높음
# - 0 기준선: 홈/원정 기대득점이 동일
# - 음수 값: 원정팀이 평균적으로 홈팀보다 기대득점이 높음 (이번 데이터에서는 거의 없음)
#
# 📌 주요 관찰 포인트:
# - 2014~2019 시즌: 평균 0.38~0.50 수준 → 홈팀이 꾸준히 xG 우위
# - 2020 시즌: 약 0.21로 급감 → COVID-19 무관중 영향 가능성
# - 2021~2023 시즌: 다시 회복하여 0.49 수준까지 상승
# - 2024 시즌: 다소 하락 → 홈 이점 감소 가능성


4-3. xG Difference by Season — Box Plot (시즌별 xG 차이 박스플롯)

In [ ]:
import plotly.express as px

# Box plot of xg_diff by season to see spread/median per season
# 시즌별 xg_diff 박스플롯 (분산/중앙값을 시즌별로 비교)
fig = px.box(
    df.dropna(subset=["xg_diff"]),
    x="season", y="xg_diff",
    title="xG Difference by Season (Home - Away)",
    labels={"season":"Season","xg_diff":"xG Difference"}
)
fig.add_hline(y=0, line_width=2, line_dash="dash", line_color="gray")
fig.update_traces(hovertemplate="Season: %{x}<br>xG diff: %{y:.3f}<extra></extra>")
fig.show()


- English

    If the median line inside the box is above 0, it means the median match in that season favored home teams in xG.
    The height of the box and whiskers show variability: a taller box means more match-to-match variation in home advantage that season.
- 한국어

    박스 안의 **중앙선(중앙값)**이 0보다 위에 있으면, 그 시즌의 중간 수준 경기에서도 홈팀이 원정보다 xG에서 유리했다는 의미.
    박스와 수염(whisker)의 길이는 시즌 내 경기 간 **편차(변동성)**를 보여줌: 박스가 클수록 시즌 내 홈 어드밴티지 차이가 경기마다 크게 변동했다는 뜻.


In [ ]:
# Step 4-3) xG Difference by Season — Box Plot
# ------------------------------------------------------------
# EN: This plot shows the distribution of match-by-match xG differences 
#     (Home xG − Away xG) for each season from 2014 to 2024.
#     - X-axis: Season
#     - Y-axis: xG Difference (Home − Away) per match
#     - Box (IQR): Middle 50% of matches
#     - Horizontal line inside box: Median xG difference for the season
#         > Median > 0 → Home teams have higher xG in the median match
#         > Median = 0 → Home and away teams have equal xG in the median match
#         > Median < 0 → Away teams have higher xG in the median match
#     - Box height & whiskers: Variability of match-to-match home advantage
#         > Taller box → Higher variability in home advantage
#         > Shorter box → More consistent xG differences
#     - Dots: Outlier matches with unusually high or low xG differences
# 
# KO: 이 그래프는 2014~2024 시즌 동안 경기별 기대득점(xG) 차이 
#     (홈 xG − 원정 xG)의 분포를 시즌별로 시각화한 것입니다.
#     - X축: 시즌
#     - Y축: 경기당 xG 차이 (홈 − 원정)
#     - 박스(IQR): 경기 결과의 중간 50% 구간
#     - 박스 안 가로선: 해당 시즌의 중앙값 xG 차이
#         > 중앙값 > 0 → 중앙 경기에서 홈팀이 원정팀보다 xG 우위
#         > 중앙값 = 0 → 중앙 경기에서 홈/원정 xG 동일
#         > 중앙값 < 0 → 중앙 경기에서 원정팀이 xG 우위
#     - 박스 높이 & 수염: 시즌 내 경기별 홈 어드밴티지 변동성
#         > 박스 높음 → 홈 어드밴티지 편차 큼
#         > 박스 낮음 → 경기별 xG 차이가 일정
#     - 점: 시즌 평균 경향에서 벗어난 특이 경기(이상치)
# 
# 📌 Observations / 주요 관찰 포인트
#     - 모든 시즌에서 중앙값이 0보다 위 → 홈팀이 평균적으로 xG 우위
#     - 2020 시즌: 중앙값 하락, 변동폭 감소 → 홈 어드밴티지 감소 (COVID-19 영향 가능성)
#     - 일부 시즌(2014, 2024)에서 극단적인 이상치 다수 존재
